In [1]:
%load_ext autoreload
%autoreload 2

In [26]:
import json
import sys
import re
import logging
import os
from pathlib import Path
from functools import partial
from itertools import chain, permutations
from uuid import uuid4

import blpapi
import ipywidgets as w
import ipyvuetify as v
import numpy as np
import pandas as pd
import QuantLib as ql
from IPython.display import display
from ipydatagrid import DataGrid, TextRenderer, BarRenderer, Expr, VegaExpr
from bqplot import LinearScale, ColorScale, OrdinalColorScale, OrdinalScale
from pandas.tseries.offsets import BDay
from plotly import graph_objects as go
from plotly.subplots import make_subplots

import bhutils
from bhutils import bhutilspy as bh
# from api.ezutils import bh, mkt_defs, ccy_cals, mx_cals, mtg_dts, gens

from gioutils.utils import parse_offset, today, get_next_n, get_bday
from api.data.base import data_path, radar_path
from gioutils.gui.base import View, ViewStore, WidgetView, Tabs
from api.gui.params import bbg_params, spot_params, fwd_params
from gioutils.blpw import BlpQuery
from gioutils.utils import RepeatedTimer
from gioutils import ezutils as ez
from gioutils.gui.pandas import millify1

from gioutils.ezutils import bh, bhs, SQL, ZCILSHelper, get_price_info, get_id, LiveInflLocal, COBInflLocal, LiveCurvesLocal, COBCurvesRemote
from api.gui.theme import bg_color, grid_style
from api.gui.pandas import DFOutput, negative_red, custom_styler_row_line, DataGridGio

pd.options.plotting.backend = "plotly"

In [27]:
from api.collectors import CollectorStoreMixin, SpotForward, Futures, Market, Mtgs, FwdMatrix, VolMatrix, ECOPred, ZCILMatrix, ECO
from api.gui.fly_grids import FlyGrid
from api.gui.editable_grid import EditableGrid
from api.gui.price_grids import PricingGrid
from api.gui.tables import SpotTable, FutureTable, GroupMktTalbe, KeyChart, MtgTable, MatTable, CustomGovtBondFutTable
from api.gui.grids import SpotGrid, FutureGrid, MtgGrid, MatGrid, GroupMktGrid, ECOGrid # GroupMktTalbe, KeyChart, MtgTable, MatTable, CustomGovtBondFutTable
from api.gui.global_config import GlobalConfig
from api.gui.monitors import MktMonitor, VolMonitor, ECOMonitor
from api.app import DataCollector
from api.gui.renderers import RendererStoreMixin, RendererComponentStoreMixin
from gioutils.gui.base import ViewStore

In [4]:
bq = BlpQuery(timeout=50000).start()

In [5]:
ccys = [
    'EUR', 
    'USD', 
    'GBP',
    # 'JPY'
]

In [7]:
pricer_grid = PricingGrid()
mm = MktMonitor()
vm = VolMonitor()
em = ECOMonitor()

In [8]:
eco_fci = ECOPred(
    X_names= [
            'GT2 Govt',
            'GT10 Govt',
            'ES1 Index',
            'CDX HY CDSI GEN 5Y SPRD Curncy',
            'CDX IG CDSI GEN 5Y SPRD Curncy',
            'CL1 Comdty',
        ],
    y_name = 'NFCIINDX Index',
    name='nfci',
    display_name='NFCI'
)

In [9]:
sf = SpotForward(spot_params=spot_params, fwd_params=fwd_params, name='sf')
pg = pricer_grid = PricingGrid(name='pg')
fut = Futures(name='fut')
mkt = Market(name='mkt')
mtgs = Mtgs(name='mtgs')
mat = FwdMatrix(name='mat'),
vol_mat = VolMatrix(name='vol_mat')
zcil_mat = ZCILMatrix(name='zcil_mat')
eco = ECO(name='eco')

In [10]:
live_infl = LiveInflLocal()
cob_infl = COBInflLocal()

In [21]:
from api.instruments.swap import RollingSwap

In [22]:
rs = RollingSwap('USD', 'USD SOFR', '2y', '5y')

In [28]:
{''COBCurvesRemote()

Init signature: COBCurvesRemote(date=None, curves=None, ccys=None, offset=None)
Docstring:      <no docstring>
File:           c:\dev\projects\gioutils\gioutils\ezutils.py
Type:           type
Subclasses:     


In [25]:
offsets = ['1w', '1m', '3m', '6m', 'ytd']

In [ ]:
gen = 'USD SOFR'

In [ ]:
cob_curves = COBCurvesRemote()

In [11]:
dc = DataCollector(
    live_infl=live_infl,
    cob_infl=cob_infl,
    ccys=ccys,
    bq=bq,
)

In [ ]:
gc = GlobalConfig(dc=dc)

In [ ]:
tabs = Tabs(
    {
        'radar': v.Container(
            fluid=True,
            children=[
                v.Row(children=[v.Col(children=[gc.view])]),
                v.Row(children=[v.Col(children=[mm.view])])
            ],
        ),
        'vol': v.Container(
            fluid=True,
            children=[vm.view]
        ),
        'eco':  v.Container(
            fluid=True,
            children=[em.view]
        ),
        'pricer': v.Container(
            fluid=True,
            children=[w.VBox([pricer_grid.box, pricer_grid.price_grid.view])],
        ),
        'monitor': pricer_grid.expr_dg.view,
    },
    force_grid_render=True,
    # tabs_kwargs={'class_': "mb-2"}
)
tabs

In [ ]:
tr = em.eco.widget._transforms

In [ ]:
em.eco.widget.transform(tr)

In [ ]:
em.eco.widget

In [ ]:
e = ECO()

In [ ]:
e.get_data(bq=bq)

In [ ]:
hist_data = e.r

In [ ]:
e.data.dtypes

In [ ]:
from py2vega.functions.type_checking import isValid

In [ ]:
def format_actual(cell):
    value = cell.value
    other = cell.metadata.data['last_actual']
    # if not isValid(value) or not isValid(other):
    #     return 'red'
    if value < other:
        return 'red'  
    else:
        return 'green'
    
def format_survey(cell):
    value = cell.value
    other = cell.metadata.data['last_actual']
    # if not isValid(value) or not isValid(other):
    #     return 'red'
    if value < other:
        return 'red'  
    else:
        return 'green'
    
    
def format_last_actual(cell):
    value = cell.value
    other = cell.metadata.data['prior_actual']
    # if not isValid(value) or not isValid(other):
    #     return 'red'
    if value < other:
        return 'red'  
    else:
        return 'green'
    
    
def format_prior_actual(cell):
    value = cell.value
    other = cell.metadata.data['prior_revision']
    # if not isValid(value) or not isValid(other):
    #     return 'red'
    if value < other:
        return 'red'  
    else:
        return 'green'

In [ ]:
from api.gui.grids import BaseGrid

In [ ]:
class ECOGrid(BaseGrid):
    
    def __init__(self, dc_key='eco_grid', **kwargs):
        kwargs['dc_key'] = dc_key
        kwargs['html_title'] = kwargs.pop('html_title', 'ECO Grid')
        super().__init__(**kwargs)
        
    def _get_data(self, dc, **kwargs):
        return dc[self.dc_key].data
    
    def _get_widget(self, data, **kwargs):
        if hasattr(self, 'widget'):
            widget_data = self.widget_data
            data = data.set_index('security').loc[[*widget_data['security']], :]
            
        
        widget = DataGridGio(
            data,
            renderers={
                'next': TextRenderer(
                    format='%d %b %Y',
                    format_type="time",
                ),
                'actual': TextRenderer(text_value=Expr(millify1), text_color=Expr(format_last)),
                'survey': TextRenderer(text_value=Expr(millify1), text_color=Expr(format_survey)),
                'last_actual': TextRenderer(text_value=Expr(millify1), text_color=Expr(format_last_actual)),
                'prior_actual': TextRenderer(text_value=Expr(millify1), text_color=Expr(format_prior_actual)),
                'last_revision': TextRenderer(text_value=Expr(millify1)),
                'prior_revision': TextRenderer(text_value=Expr(millify1)),
                'rank': BarRenderer(
                    bar_color=ColorScale(min=0, max=120),
                    bar_value=LinearScale(min=0, max=100),
                    text_value=VegaExpr("' '"),
                )
            },
            column_widths={
                'security': 150,
                'country': 70,
                'name': 250,
                'next': 100,
                'survey': 70,
                'last_actual': 90,
                'prior_actual': 95,
                'last_revision': 100,
                'prior_revision': 105,
            },
            index_name='',
            selection_mode='cell',
            editable=True
        )
        
        return widget
        
        

In [ ]:
e.data

In [ ]:
widget = DataGridGio(
    data,
    renderers={
        'next': TextRenderer(
            format='%d %b %Y',
            format_type="time",
        ),
        'actual': TextRenderer(text_value=Expr(millify1), text_color=Expr(format_last)),
        'survey': TextRenderer(text_value=Expr(millify1), text_color=Expr(format_survey)),
        'last_actual': TextRenderer(text_value=Expr(millify1), text_color=Expr(format_last_actual)),
        'prior_actual': TextRenderer(text_value=Expr(millify1), text_color=Expr(format_prior_actual)),
        'last_revision': TextRenderer(text_value=Expr(millify1)),
        'prior_revision': TextRenderer(text_value=Expr(millify1)),
        'rank': BarRenderer(
            bar_color=ColorScale(min=0, max=120),
            bar_value=LinearScale(min=0, max=100),
            text_value=VegaExpr("' '"),
        )
    },
    column_widths={
        'security': 150,
        'country': 70,
        'name': 250,
        'next': 100,
        'survey': 70,
        'last_actual': 90,
        'prior_actual': 95,
        'last_revision': 100,
        'prior_revision': 105,
    },
    index_name='',
    selection_mode='cell',
    editable=True
)

In [ ]:
e.data

In [ ]:
class LiveFXSpot:
    def __init__(self, **kwargs):

        self.mkt = bh.bhCreateLiveFxMarket(
            market="Default",
            quantserver=SQL.Quants.server,
            quantdb=SQL.Quants.db,
            mktdatahost="localhost",
            mktdataport=60000
        )
    
    def get_snap(self, **kwargs):
        return bh.bhFxMarketSnap(market=self.mkt)
    
    @property
    def snap(self):
        return self.get_snap()

In [ ]:
def get_fx_curves_from_snap(curve_snap):
    curve_snap_df = curve_snap.py
    fx_curves = curve_snap_df.loc[
        curve_snap_df['name'].str.endswith('.FX'),
        ['currency', 'name', 'curve']
    ]
    return fx_curves

def get_ccy_pairs(curve_snap=None, fx_curves=None, ccys=None, usdusd=True):
    if fx_curves is None:
        fx_curves = get_fx_curves_from_snap(curve_snap)
        
    if ccys:
        fx_curves = fx_curves.query(f'currency in {ccys}')
    ccys_ = [*fx_curves['currency']]
    ccypairs = [*permutations(ccys_, 2)]
    if usdusd:
        ccypairs = [('USD', 'USD'), *ccypairs]
    ccypairs_str = [''.join(pair) for pair in ccypairs]
    return ccypairs_str

In [ ]:
class LiveFXFwd:
    def __init__(self, ccys=None, date=None, vol_update_freq=60, interp='sabr', **kwargs):
        self.ccys = ccys
        self.date = date or get_bday(today())
        self._last_update = None
        self.vol_update_freq = vol_update_freq
        self.interp = interp
        
    def get_snap(self, curve_snap, fx_snap, **kwargs):
        fx_opt_mkt = self.get_fx_opt_mkt(curve_snap, fx_snap)
        self.fx_opt_mkt = fx_opt_mkt
        return fx_opt_mkt
    
    def get_fx_fwd_mkt(self, curve_snap, fx_snap):
        date = self.date
        ccys = self.ccys
        
        fx_curves = get_fx_curves_from_snap(curve_snap)
        ccypairs_str = get_ccy_pairs(fx_curves=fx_curves)
        
        fx_spot_snap_df = lfx.snap.py
        spot_rates = fx_spot_snap_df.query(f'ccypair in {ccypairs_str} and rate.notna()')
        mkt = bh.bhCreateFxFwdMarket2(
            asofdate=date,
            mktdefs=bhs.mkt_defs,
            cals=bhs.ccy_cals,
            ccys=[*fx_curves['currency']],
            crvnames=[*fx_curves['name']],
            curves=[*fx_curves['curve']],
            ccypairs=[*spot_rates['ccypair']],
            spotrates=[*spot_rates['rate']],
            # ccyor=None,
            # crvnamesor=None,
            # curvesor=None,
            # instrumentor=None,
            # scalingfactors=None,
            # csacategoryor=None,
        )
        return mkt
    
    def get_vols_data(self, ccypairs):
        date = self.date
        vol_data = bh.bhSnapFxVolData(
            ccypairs=ccypairs,
            asofdate=date,
            fxmktdefs=bhs.fx_mkt_defs,
            cals=bhs.ccy_cals,
            mktdefs=bhs.mkt_defs,
            date=date,
            snapserver=SQL.Snap.server,
            snapdb=SQL.Snap.db,
            maxtenor="5Y",
        )
        return vol_data
    
    def get_vol_wgt(self, ccypairs):
        date = self.date
        vol_wgt = bh.bhSnapFxVolWeights(
            ccypairs=ccypairs,
            datadate=date,
            server=SQL.Snap.server,
            database=SQL.Snap.db,
        )
        return vol_wgt
    
    def get_fx_opt_mkt(self, curve_snap, fx_snap):
        last_update = self._last_update
        self._last_update = pd.Timestamp.now()
        passed = (self._last_update - last_update).seconds / 60 if last_update else self.vol_update_freq 
        interp = self.interp
        
        mkt = self.get_fx_fwd_mkt(curve_snap, fx_snap)
        self.fwd_mkt = mkt
        ccypairs = get_ccy_pairs(curve_snap, usdusd=False)
        date = self.date 
        
        if passed >= self.vol_update_freq:
            vol_data = self.get_vols_data(ccypairs)
            vol_wgt = self.get_vol_wgt(ccypairs)
            slv_params = self.get_slv_params(ccypairs)

            self._vol_data = vol_data
            self._vol_wgt = vol_wgt
            self._slv_params = slv_params
        else:
            vol_data = self._vol_data
            vol_wgt = self._vol_wgt
            slv_params = self._slv_params
        
        vols = [*vol_data.py['voldata']]
        interps = [interp] * len(vols)

        fx_options_mkt = bh.bhCreateFxOptionMarket3(
            fwdmkt=mkt,
            vols=vols,
            smilesliceinterp=interps,
            fxmktdefs=bhs.fx_mkt_defs,
            useweighteddays=True,
            volslicetype=None,
            hestonquadraticslvs=[*slv_params.py['voldata']],
            dayweighttimeseries=[*vol_wgt.py['timeseries']],
            fixings=None,
        )
        return fx_options_mkt
    
    def get_slv_params(self, ccypairs):
        date = self.date
        slv_params = bh.bhSnapFxSLVParameters(
            ccypairs=ccypairs,
            asofdate=date,
            cals=bhs.ccy_cals,
            mktdefs=bhs.mkt_defs,
            fxmktdefs=bhs.fx_mkt_defs,
            date=date,
            snapserver=SQL.Snap.server,
            snapdb=SQL.Snap.db,
            category=None,
        )
        return slv_params
    

In [ ]:
ccys = [
    'USD',
    'EUR',
    'AUD',
    'GBP',
    'NZD',
    'CAD',
    'MXN',
]

In [ ]:
lfx = LiveFXSpot()
lfxfwd = LiveFXFwd(ccys=ccys)
self = lfxfwd

In [ ]:
curve_snap = dc.live_curves.cached_snap
fx_snap = lfx.snap

In [ ]:
snap = self.get_snap(curve_snap, fx_snap)

In [ ]:
mkt = lfxfwd.get_snap(curve_snap, fx_snap)
ccypairs = get_ccy_pairs(curve_snap, usdusd=False)
date = self.date 

In [ ]:
vol_data = bh.bhSnapFxVolData(
    ccypairs=ccypairs,
    asofdate=date,
    fxmktdefs=bhs.fx_mkt_defs,
    cals=bhs.ccy_cals,
    mktdefs=bhs.mkt_defs,
    date=date,
    snapserver=SQL.Snap.server,
    snapdb=SQL.Snap.db,
    maxtenor="5Y",
)

In [ ]:
vol_wgt = bh.bhSnapFxVolWeights(
    ccypairs=ccypairs,
    datadate=date,
    server=SQL.Snap.server,
    database=SQL.Snap.db,
)

In [ ]:
slv_params = bh.bhSnapFxSLVParameters(
    ccypairs=ccypairs,
    asofdate=date,
    cals=bhs.ccy_cals,
    mktdefs=bhs.mkt_defs,
    fxmktdefs=bhs.fx_mkt_defs,
    date=date,
    snapserver=SQL.Snap.server,
    snapdb=SQL.Snap.db,
    category=None,
)

In [ ]:
interp = 'sabr'

In [ ]:
vols = [*vol_data.py['voldata']]
interps = [interp] * len(vols)

fx_options_mkt = bh.bhCreateFxOptionMarket3(
    fwdmkt=mkt,
    vols=vols,
    smilesliceinterp=interps,
    fxmktdefs=bhs.fx_mkt_defs,
    useweighteddays=True,
    volslicetype=None,
    hestonquadraticslvs=[*slv_params.py['voldata']],
    dayweighttimeseries=[*vol_wgt.py['timeseries']],
    fixings=None,
)

In [ ]:

[*vol_wgt.py['timeseries']]
[*vol_wgt.py['timeseries']]

In [ ]:
curve_snap_df = curve_snap.py

In [ ]:
bh.bhSnapFxSLVParameters?

In [ ]:
from gioutils.ezutils import ZCILSHelper

In [ ]:
from api.collectors import BaseCollector, CollectorStoreMixin

In [ ]:
from api.collectors import get_mtgs_df

In [ ]:
snaps = dc.get_snaps()

In [ ]:
mtgs.get_data(snaps, bq=bq)

In [ ]:
df = get_mtgs_df('USD', parse=False)

In [ ]:
df

In [ ]:
df['MeetingDate'].replace('fixing', get_bday(today()))

In [ ]:
w.HBox(
    [
        w.Dropdown(options=['call', 'put']);
        layout
    ]
)